In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv(r"C:\Users\gratt\Desktop\emsData\raceAgeSexAcuityTrauma.csv", index_col='PcrKey', dtype=int)

In [5]:
df

,hospCapability,initialPtAcuity,black,white,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander,ageinyear,sex
PcrKey,,,,,,,,,,
118170851,0,2,0,0,1,0,0,0,16,1
118170860,0,3,0,1,0,0,0,0,49,1
118170980,0,3,0,1,0,0,0,0,30,0
118171192,0,3,1,0,0,0,0,0,58,1
118171212,0,2,0,1,0,0,0,0,41,0
...,...,...,...,...,...,...,...,...,...,...
168637433,0,2,0,1,0,0,0,0,45,1
168637452,0,3,0,1,0,0,0,0,15,0
168637590,0,3,0,1,0,0,0,0,51,1


In [10]:
df['initialPtAcuity'].value_counts(normalize=True)

2    0.650441
3    0.292272
4    0.056361
1    0.000927
Name: initialPtAcuity, dtype: float64

In [11]:
df['black'].value_counts(normalize=True)

0    0.821114
1    0.177791
2    0.001096
Name: black, dtype: float64

In [6]:
crit = df[ df['initialPtAcuity'] == 4]

In [7]:
crit

,hospCapability,initialPtAcuity,black,white,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander,ageinyear,sex
PcrKey,,,,,,,,,,
118171324,0,4,1,0,0,0,0,0,19,1
118171963,1,4,0,1,0,0,0,0,37,1
118172225,0,4,0,1,0,0,0,0,25,1
118172592,0,4,0,1,0,0,0,0,91,1
118255842,1,4,0,0,1,0,0,0,27,0
...,...,...,...,...,...,...,...,...,...,...
168631386,0,4,0,0,0,0,0,1,80,1
168631542,1,4,0,1,0,0,0,0,9,1
168631858,1,4,0,1,0,0,0,0,21,0


In [8]:
# % of pts critically injured
133164 / 828957

0.16064041922560518

In [20]:
# patients critically injured who were transported to a trauma center
crit['hospCapability'].value_counts(normalize=True)

0          0.361779
1          0.253973
0          0.153623
1          0.101867
7701003    0.022776
9908007    0.020937
7701003    0.018451
9908007    0.015717
9908021    0.010746
7701001    0.008951
9908021    0.008110
7701001    0.007442
9908023    0.004115
9908023    0.003304
9908025    0.001337
9908005    0.000939
9908025    0.000864
9908011    0.000751
9908027    0.000608
9908011    0.000601
9908005    0.000511
9908017    0.000466
9908017    0.000443
9908003    0.000406
9908003    0.000345
9908027    0.000285
9908031    0.000113
9908033    0.000113
9908033    0.000113
9908031    0.000090
9908019    0.000045
9908009    0.000045
9908001    0.000030
9908029    0.000030
9908029    0.000023
9908019    0.000015
9908001    0.000015
9908009    0.000015
Unknown    0.000008
Name: hospCapability, dtype: float64